# Détection et reconnaissance des panneaux de signalisation

Dans le monde de l'IA des grandes entreprises comme Tesla, Uber, Google, Mercedes-Benz, etc. travaillent sur des véhicules autonomes et des voitures autonomes. Pour atteindre la précision de cette technologie, les véhicules doivent être capables d'interpréter les panneaux de signalisation et de prendre des décisions en conséquence. Votre objectif est de construire un modèle qui permet de réaliser cette tâche.

Une interface graphique est demandée:

    Qui permet de détecter dans une image les panneaux de signalisation et les reconnaitre (Afficher le label en Français).
    Qui permet de détecter tous les panneaux dans une vidéo les reconnaitre et les afficher avec leurs labels.


# Modalités d'évaluation

    Un rapport sur le projet réalisé qui explique les différentes étapes du code

    Description des données

    Présentation de l'architecture utilisée

    Conclusion (avantages et inconvénients, concurrents, recommandations…)

    Revue de code avec le formateur.



# Étape 1: Explorez l'ensemble de données


Créez un fichier de script Python et nommez-le traffic_signs.py dans le dossier du projet.

Notre approche pour créer ce modèle de classification des panneaux de signalisation est présentée en quatre étapes:

Explorez l'ensemble de données
Construire un modèle CNN
Former et valider le modèle
Tester le modèle avec un jeu de données de test

Notre dossier «train» contient 43 dossiers représentant chacun une classe différente. La plage du dossier va de 0 à 42. Avec l'aide du module OS, nous parcourons toutes les classes et ajoutons des images et leurs étiquettes respectives dans la liste des données et des étiquettes.

La bibliothèque PIL est utilisée pour ouvrir le contenu de l'image dans un tableau.

In [1]:
# Importation des bibliothéques
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

In [2]:
# Stockage des labels et de données  dans une liste 
data = []
labels = []
classes = 43
# getcwd() retourne le dossier de travail courant
cur_path = os.getcwd()

In [3]:
# Récupération des données et des labels
for i in range(classes):
    path = os.path.join(cur_path,'train',str(i))
    images = os.listdir(path)
    for a in images:
        try:
            image = Image.open(path + '\\'+ a)
            image = image.resize((30,30))
            image = np.array(image)
            data.append(image)
            labels.append(i)
        except:
            print("Error loading image")

Error loading image


In [4]:
# Conversion des listes en tableau np.array
data = np.array(data)
labels = np.array(labels)
print(data.shape, labels.shape)

(50012, 30, 30, 3) (50012,)


In [5]:
#Fractionnement des données en jeu de test et de données
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(40009, 30, 30, 3) (10003, 30, 30, 3) (40009,) (10003,)


In [6]:
# Encodage des labels
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

In [7]:
# Construction du modèle
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))


In [ ]:
#Compilation du modèle
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
epochs = 15
history = model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_data=(X_test, y_test))
model.save("my_model.h5")


Epoch 1/15
1251/1251 [==============================] - 505s 398ms/step - loss: 2.9320 - accuracy: 0.4051 - val_loss: 0.4332 - val_accuracy: 0.8920
Epoch 2/15
1251/1251 [==============================] - 8504s 7s/step - loss: 0.6961 - accuracy: 0.7994 - val_loss: 0.2137 - val_accuracy: 0.9423
Epoch 3/15
1239/1251 [============================>.] - ETA: 18s - loss: 0.4878 - accuracy: 0.8565

In [ ]:
# Affichage de  graphiques pour plus de précision
plt.figure(0)
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()
plt.figure(1)
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
# Test de la précision sur l'ensemble de données de test
from sklearn.metrics import accuracy_score
y_test = pd.read_csv('Test.csv')
labels = y_test["ClassId"].values
imgs = y_test["Path"].values
data=[]

In [ ]:
# Ouverture des images(données) et redimensionnement
for img in imgs:
    image = Image.open(img)
    image = image.resize((30,30))
    data.append(np.array(image))
X_test=np.array(data)
pred = model.predict_classes(X_test)

In [ ]:
# Précision avec les données de test
from sklearn.metrics import accuracy_score
print(accuracy_score(labels, pred))


In [ ]:
# Sauvegarde du modèle
model.save('traffic_classifier.h5')

In [ ]:
# Précision du modèle à 94 % 
import tkinter as tk
from tkinter import filedialog
from tkinter import *
from PIL import ImageTk, Image
import numpy
# Chargement du modèle
from keras.models import load_model
model = load_model('traffic_classifier.h5')


In [ ]:
# Dictionnaire de tous les labels
classes = { 1:'Limite de vitesse (20km/h)',
            2:'Limite de vitesse (30km/h)', 
            3:'Limite de vitesse (50km/h)', 
            4:'Limite de vitesset (60km/h)', 
            5:'Limite de vitesse (70km/h)', 
            6:'Limite de vitesse (80km/h)', 
            7:'Fin de limite de vistesse (80km/h)', 
            8:'Limite de vitesse (100km/h)', 
            9:'Limite de vitesse (120km/h)', 
            10:'Dépassement interdit', 
            11:'Dépassement interdit pour les plus de 3,5 tonnes', 
            12:'Priorité à droite', 
            13:'Route prioritaire', 
            14:'Cédez le passage', 
            15:'Stop', 
            16:'Véhicule interdit', 
            17:'Interdit au plus de 3 tonnes', 
            18:'No entry', 
            19:'Danger', 
            20:'Virage à gauche dangereux', 
            21:'Virage à droite dangeureux', 
            22:'Virages', 
            23:"Dos d'âne", 
            24:'Route glissante', 
            25:'Route serrée à droite', 
            26:'Route en travaux', 
            27:'Feux tricolore', 
            28:'Passage piétons', 
            29:"Passage d'enfants", 
            30:'Piste cyclable', 
            31:'Attention à la neige et au gel',
            32:"Traversée d'animaux", 
            33:'Fin de toutes les interdictions', 
            34:'Tourner obligatoirement à droite', 
            35:'Tourner obligatoire à gauche', 
            36:'Aller obligatoirement tout droit', 
            37:'Aller à droite ou tout droit', 
            38:'Aller à gauche ou tout droit', 
            39:'Restez à droite', 
            40:'restez à gauche', 
            41:'Rond-point', 
            42:'Fin de dépassement', 
            43:'Fin de dépassement pour les plus de 3,5 tonnes' }

In [ ]:
# Initialisation de la fenêtre de l'application
top=tk.Tk()
top.geometry('800x600')
top.title('Traffic sign classification')
top.configure(background='#CDCDCD')
label=Label(top,background='#CDCDCD', font=('arial',15,'bold'))
sign_image = Label(top)


In [ ]:
# Fonction de classification
def classify(file_path):
    global label_packed
    image = Image.open(file_path)
    image = image.resize((30,30))
    image = numpy.expand_dims(image, axis=0)
    image = numpy.array(image)
    pred = model.predict_classes([image])[0]
    sign = classes[pred+1]
    print(sign)
    label.configure(foreground='#011638', text=sign) 


In [ ]:
# Fonction pour le bouton afin de classifier le panneau(image)
def show_classify_button(file_path):
    classify_b=Button(top,text="Classify Image",command=lambda: classify(file_path),padx=10,pady=5)
    classify_b.configure(background='#364156', foreground='white',font=('arial',10,'bold'))
    classify_b.place(relx=0.79,rely=0.46)

In [ ]:
# Fonction de chargement d'image    
def upload_image():
    try:
        file_path=filedialog.askopenfilename()
        uploaded=Image.open(file_path)
        uploaded.thumbnail(((top.winfo_width()/2.25),(top.winfo_height()/2.25)))
        im=ImageTk.PhotoImage(uploaded)
        sign_image.configure(image=im)
        sign_image.image=im
        label.configure(text='')
        show_classify_button(file_path)
    except:
        pass


In [ ]:
upload=Button(top,text="Upload an image",command=upload_image,padx=10,pady=5)
upload.configure(background='#364156', foreground='white',font=('arial',10,'bold'))
upload.pack(side=BOTTOM,pady=50)
sign_image.pack(side=BOTTOM,expand=True)
label.pack(side=BOTTOM,expand=True)
heading = Label(top, text="Know Your Traffic Sign",pady=20, font=('arial',20,'bold'))
heading.configure(background='#CDCDCD',foreground='#364156')
heading.pack()
top.mainloop()